Importing Symbols

In [22]:
import datetime
import requests
import bs4
import pymysql
pymysql.install_as_MySQLdb()
import MySQLdb as mdb
import pandas as pd

In [7]:
def obtain_parse_wiki_snp500():
    """
    Download and parse the Wikipedia list of S&P 500 
    constituents using requests and BeautifulSoup.

    Returns a list of tuples to add to MySQL.
    """
    # Stores the current time, for the created_at record
    now = datetime.datetime.utcnow()

    # Use requests and BeautifulSoup to download the 
    # list of S&P 500 companies and obtain the symbol table
    response = requests.get(
        "http://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    )
    soup = bs4.BeautifulSoup(response.text, features="html.parser")

    # This selects the first table, using CSS Selector syntax
    # and then ignores the header row ([1:])
    symbolslist = soup.select('table')[0].select('tr')[1:]

    # Obtain the symbol information for each 
    # row in the S&P 500 constituent table
    symbols = []
    for i, symbol in enumerate(symbolslist):
        tds = symbol.select('td')
        symbols.append(
            (
                tds[0].select('a')[0].text.strip(),  # Ticker
                'stock', 
                tds[1].text.strip(),  # Name
                tds[3].text.strip(),  # Sector
                'USD', now, now
            ) 
        )
    return symbols


In [14]:
def insert_snp500_symbols(symbols):
    """
    Insert the S&P500 symbols into the MySQL database.
    """
    # Connect to the MySQL instance
    db_host = 'localhost'
    db_user = 'sec_user'
    db_pass = '1120'
    db_name = 'securities_master'
    con = mdb.connect(
        host=db_host, user=db_user, passwd=db_pass, db=db_name
    )

    # Create the insert strings
    column_str = """ticker, instrument, name, sector, 
                 currency, created_date, last_updated_date
                 """
    insert_str = ("%s, " * 7)[:-2]
    final_str = "INSERT INTO symbol (%s) VALUES (%s)" % \
        (column_str, insert_str)

    # Using the MySQL connection, carry out 
    # an INSERT INTO for every symbol
    with con: 
        cur = con.cursor()
        cur.executemany(final_str, symbols)

In [8]:
symbols = obtain_parse_wiki_snp500()

# Example: Print the first symbol
print(symbols[0])

# Example: Insert into your database
for symbol in symbols:
    print(symbol)  # Debugging: see each tuple before inserting
    # Insert `symbol` into your database (assuming you've set up the connection)


('MMM', 'stock', '3M', 'Industrial Conglomerates', 'USD', datetime.datetime(2024, 8, 13, 12, 45, 2, 574801), datetime.datetime(2024, 8, 13, 12, 45, 2, 574801))
('MMM', 'stock', '3M', 'Industrial Conglomerates', 'USD', datetime.datetime(2024, 8, 13, 12, 45, 2, 574801), datetime.datetime(2024, 8, 13, 12, 45, 2, 574801))
('AOS', 'stock', 'A. O. Smith', 'Building Products', 'USD', datetime.datetime(2024, 8, 13, 12, 45, 2, 574801), datetime.datetime(2024, 8, 13, 12, 45, 2, 574801))
('ABT', 'stock', 'Abbott', 'Health Care Equipment', 'USD', datetime.datetime(2024, 8, 13, 12, 45, 2, 574801), datetime.datetime(2024, 8, 13, 12, 45, 2, 574801))
('ABBV', 'stock', 'AbbVie', 'Biotechnology', 'USD', datetime.datetime(2024, 8, 13, 12, 45, 2, 574801), datetime.datetime(2024, 8, 13, 12, 45, 2, 574801))
('ACN', 'stock', 'Accenture', 'IT Consulting & Other Services', 'USD', datetime.datetime(2024, 8, 13, 12, 45, 2, 574801), datetime.datetime(2024, 8, 13, 12, 45, 2, 574801))
('ADBE', 'stock', 'Adobe Inc.'

In [19]:
import pymysql
import datetime
import bs4
import requests

# Database connection details
db_host = 'localhost'
db_user = 'sec_user'
db_pass = '1120'
db_name = 'securities_master'

# Function to obtain and parse symbols
def obtain_parse_wiki_snp500():
    """
    Download and parse the Wikipedia list of S&P 500 
    constituents using requests and BeautifulSoup.

    Returns a list of tuples to add to MySQL.
    """
    # Stores the current time, for the created_at record
    now = datetime.datetime.utcnow()

    # Use requests and BeautifulSoup to download the 
    # list of S&P 500 companies and obtain the symbol table
    response = requests.get(
        "http://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    )
    soup = bs4.BeautifulSoup(response.text, features="html.parser")

    # This selects the first table, using CSS Selector syntax
    # and then ignores the header row ([1:])
    symbolslist = soup.select('table')[0].select('tr')[1:]

    # Obtain the symbol information for each 
    # row in the S&P 500 constituent table
    symbols = []
    for symbol in symbolslist:
        tds = symbol.select('td')
        symbols.append(
            (
                tds[0].select('a')[0].text.strip(),  # Ticker
                'stock', 
                tds[1].text.strip(),  # Name
                tds[3].text.strip(),  # Sector
                'USD', now, now
            )
        )
    return symbols

# Main script to connect to the database and insert symbols
try:
    # Establish a database connection
    con = pymysql.connect(host=db_host, user=db_user, password=db_pass, database=db_name)
    with con.cursor() as cur:
        # Obtain and parse the S&P 500 symbols
        symbols = obtain_parse_wiki_snp500()

        # Insert the symbols into the 'symbol' table
        for symbol in symbols:
            print(symbol)  # Debugging: check the content of the symbol tuple
            cur.execute(
                "INSERT INTO symbol (ticker, instrument, name, sector, currency, created_date, last_updated_date) "
                "VALUES (%s, %s, %s, %s, %s, %s, %s)", 
                symbol
            )

        # Commit the transaction to save changes to the database
        con.commit()
        print("Symbols successfully added to the database.")
except pymysql.MySQLError as e:
    print(f"Error: {e}")
finally:
    # Ensure the connection is closed even if an error occurs
    if con:
        con.close()


('MMM', 'stock', '3M', 'Industrial Conglomerates', 'USD', datetime.datetime(2024, 8, 13, 13, 7, 16, 337196), datetime.datetime(2024, 8, 13, 13, 7, 16, 337196))
('AOS', 'stock', 'A. O. Smith', 'Building Products', 'USD', datetime.datetime(2024, 8, 13, 13, 7, 16, 337196), datetime.datetime(2024, 8, 13, 13, 7, 16, 337196))
('ABT', 'stock', 'Abbott', 'Health Care Equipment', 'USD', datetime.datetime(2024, 8, 13, 13, 7, 16, 337196), datetime.datetime(2024, 8, 13, 13, 7, 16, 337196))
('ABBV', 'stock', 'AbbVie', 'Biotechnology', 'USD', datetime.datetime(2024, 8, 13, 13, 7, 16, 337196), datetime.datetime(2024, 8, 13, 13, 7, 16, 337196))
('ACN', 'stock', 'Accenture', 'IT Consulting & Other Services', 'USD', datetime.datetime(2024, 8, 13, 13, 7, 16, 337196), datetime.datetime(2024, 8, 13, 13, 7, 16, 337196))
('ADBE', 'stock', 'Adobe Inc.', 'Application Software', 'USD', datetime.datetime(2024, 8, 13, 13, 7, 16, 337196), datetime.datetime(2024, 8, 13, 13, 7, 16, 337196))
('AMD', 'stock', 'Advance

In [20]:
if __name__ == "__main__":
    # Obtain a database connection to the MySQL instance
    db_host = 'localhost'
    db_user = 'sec_user'
    db_pass = '1120'
    db_name = 'securities_master'
try:
    con = pymysql.connect(host=db_host, user=db_user, password=db_pass, database=db_name)
    print("Connected to the database successfully!")
except pymysql.MySQLError as e:
    print(f"Error {e.args[0]}: {e.args[1]}")

    # Select all of the historic Google adjusted close data
    sql = """SELECT dp.price_date, dp.adj_close_price
             FROM symbol AS sym
             INNER JOIN daily_price AS dp
             ON dp.symbol_id = sym.id
             WHERE sym.ticker = 'GOOG'
             ORDER BY dp.price_date ASC;"""

    # Create a pandas dataframe from the SQL query
    goog = pd.read_sql_query(sql, con=con, index_col='price_date')    

    # Output the dataframe tail
    print(goog.tail())


Connected to the database successfully!


In [23]:
    # Select all of the historic Google adjusted close data
    sql = """SELECT dp.price_date, dp.adj_close_price
             FROM symbol AS sym
             INNER JOIN daily_price AS dp
             ON dp.symbol_id = sym.id
             WHERE sym.ticker = 'GOOG'
             ORDER BY dp.price_date ASC;"""

    # Create a pandas dataframe from the SQL query
    goog = pd.read_sql_query(sql, con=con, index_col='price_date')    

    # Output the dataframe tail
    print(goog.tail())

/tmp/ipykernel_3639/1002139558.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  goog = pd.read_sql_query(sql, con=con, index_col='price_date')


Empty DataFrame
Columns: [adj_close_price]
Index: []


In [24]:
pip install sqlalchemy


     |████████████████████████████████| 3.1 MB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 667 kB 2.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql

# Database connection details
db_host = 'localhost'
db_user = 'sec_user'
db_pass = '1120'
db_name = 'securities_master'

# Create SQLAlchemy engine
engine = create_engine(f"mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}")

# SQL query to get Google adjusted close prices
sql = """SELECT dp.price_date, dp.adj_close_price
         FROM symbol AS sym
         INNER JOIN daily_price AS dp
         ON dp.symbol_id = sym.id
         WHERE sym.ticker = 'GOOG'
         ORDER BY dp.price_date ASC;"""

# Create a pandas dataframe from the SQL query
goog = pd.read_sql_query(sql, con=engine, index_col='price_date')

# Output the dataframe tail
print(goog.head())


Empty DataFrame
Columns: [adj_close_price]
Index: []


In [1]:
import yfinance as yf
import pymysql

def get_daily_historic_data_yahoo(ticker):
    prices = None
    try:
        ticker_data = yf.Ticker(ticker)
        prices = ticker_data.history(period="max")
        prices = prices[['Open', 'High', 'Low', 'Close', 'Volume']]
        prices.reset_index(inplace=True)
    except Exception as e:
        print(f"Could not download Yahoo data: {e}")
    return prices

# Database connection and other code here...
# Obtain a database connection to the MySQL instance
db_host = 'localhost'
db_user = 'sec_user'
db_pass = '1120'
db_name = 'securities_master'

try:
    con = pymysql.connect(host=db_host, user=db_user, password=db_pass, database=db_name)
    print("Connected to the database successfully!")

    # Example loop over tickers
    for i, t in enumerate(tickers):
        print(f"Adding data for {t[1]}: {i+1} out of {len(tickers)}")
        yf_data = get_daily_historic_data_yahoo(t[1])
        if yf_data is not None:
            # Insert yf_data into your database
            pass
        else:
            print(f"Skipping {t[1]} due to data download failure.")
except pymysql.MySQLError as e:
    print(f"Database error: {e}")
finally:
    if con:
        con.close()


Connected to the database successfully!


NameError: name 'tickers' is not defined

In [4]:
from __future__ import print_function

import datetime
import warnings
import pymysql
import requests
import yfinance as yf
import pandas as pd

# Database connection details
db_host = 'localhost'
db_user = 'sec_user'
db_pass = '1120'
db_name = 'securities_master'

def obtain_list_of_db_tickers(con):
    """
    Obtains a list of the ticker symbols in the database.
    """
    with con.cursor() as cur: 
        cur.execute("SELECT id, ticker FROM symbol")
        data = cur.fetchall()
        return [(d[0], d[1]) for d in data]

def get_daily_historic_data_yahoo(ticker, start_date="2000-01-01"):
    """
    Obtains data from Yahoo Finance using yfinance and returns a DataFrame.

    ticker: Yahoo Finance ticker symbol, e.g., "GOOG" for Google, Inc.
    start_date: Start date in "YYYY-MM-DD" format.
    """
    try:
        ticker_data = yf.Ticker(ticker)
        df = ticker_data.history(start=start_date, end=datetime.date.today().strftime("%Y-%m-%d"))

        # Check if 'Adj Close' is available, otherwise fallback to 'Close'
        if 'Adj Close' not in df.columns:
            print(f"Warning: 'Adj Close' not available for {ticker}. Using 'Close' instead.")
            df['Adj Close'] = df['Close']
        
        df = df[['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']]
        df.reset_index(inplace=True)
        df['Date'] = pd.to_datetime(df['Date'])
        return df
    except Exception as e:
        print(f"Could not download Yahoo data for {ticker}: {e}")
        return None

def insert_daily_data_into_db(con, data_vendor_id, symbol_id, daily_data):
    """
    Takes a DataFrame of daily data and adds it to the
    MySQL database. Appends the vendor ID and symbol ID to the data.
    """
    # Create the time now
    now = datetime.datetime.utcnow()

    # Amend the data to include the vendor ID and symbol ID
    daily_data = [
        (data_vendor_id, symbol_id, row['Date'], now, now,
        row['Open'], row['High'], row['Low'], row['Close'], 
        row['Volume'], row['Adj Close']) 
        for index, row in daily_data.iterrows()
    ]

    # Create the insert strings
    column_str = """data_vendor_id, symbol_id, price_date, created_date, 
                 last_updated_date, open_price, high_price, low_price, 
                 close_price, volume, adj_close_price"""
    insert_str = ("%s, " * 11)[:-2]
    final_str = "INSERT INTO daily_price (%s) VALUES (%s)" % \
        (column_str, insert_str)

    # Using the MySQL connection, carry out an INSERT INTO for every symbol
    try:
        with con.cursor() as cur:
            cur.executemany(final_str, daily_data)
        con.commit()
    except pymysql.MySQLError as e:
        print(f"Error inserting data into the database: {e}")
        con.rollback()

if __name__ == "__main__":
    # This ignores the warnings regarding Data Truncation
    warnings.filterwarnings('ignore')

    try:
        # Establish a database connection
        con = pymysql.connect(host=db_host, user=db_user, password=db_pass, database=db_name)
        print("Connected to the database successfully!")
        
        # Loop over the tickers and insert the daily historical data into the database
        tickers = obtain_list_of_db_tickers(con)
        lentickers = len(tickers)
        for i, t in enumerate(tickers):
            print(f"Adding data for {t[1]}: {i+1} out of {lentickers}")
            yf_data = get_daily_historic_data_yahoo(t[1])
            if yf_data is not None:
                insert_daily_data_into_db(con, '1', t[0], yf_data)
            else:
                print(f"Skipping {t[1]} due to data retrieval failure.")
        print("Successfully added Yahoo Finance pricing data to DB.")

    except pymysql.MySQLError as e:
        print(f"Database error: {e}")
    
    finally:
        if con and con.open:
            con.close()
            print("Database connection closed.")


Connected to the database successfully!
Adding data for MMM: 1 out of 503
Adding data for AOS: 2 out of 503
Adding data for ABT: 3 out of 503
Adding data for ABBV: 4 out of 503
Adding data for ACN: 5 out of 503
Adding data for ADBE: 6 out of 503
Adding data for AMD: 7 out of 503
Adding data for AES: 8 out of 503
Adding data for AFL: 9 out of 503
Adding data for A: 10 out of 503
Adding data for APD: 11 out of 503
Adding data for ABNB: 12 out of 503
Adding data for AKAM: 13 out of 503
Adding data for ALB: 14 out of 503
Adding data for ARE: 15 out of 503
Adding data for ALGN: 16 out of 503
Adding data for ALLE: 17 out of 503
Adding data for LNT: 18 out of 503
Adding data for ALL: 19 out of 503
Adding data for GOOGL: 20 out of 503
Adding data for GOOG: 21 out of 503
Adding data for MO: 22 out of 503
Adding data for AMZN: 23 out of 503
Adding data for AMCR: 24 out of 503
Adding data for AEE: 25 out of 503
Adding data for AAL: 26 out of 503
Adding data for AEP: 27 out of 503
Adding data for 

$BRK.B: possibly delisted; No timezone found


Adding data for BRK.B: 63 out of 503
Adding data for BBY: 64 out of 503
Adding data for BIO: 65 out of 503
Adding data for TECH: 66 out of 503
Adding data for BIIB: 67 out of 503
Adding data for BLK: 68 out of 503
Adding data for BX: 69 out of 503
Adding data for BA: 70 out of 503
Adding data for BKNG: 71 out of 503
Adding data for BWA: 72 out of 503
Adding data for BSX: 73 out of 503
Adding data for BMY: 74 out of 503
Adding data for AVGO: 75 out of 503
Adding data for BR: 76 out of 503
Adding data for BRO: 77 out of 503
Adding data for BF.B: 78 out of 503


$BF.B: possibly delisted; No price data found  (1d 2000-01-01 -> 2024-08-13)


$BF.B: possibly delisted; No price data found  (1d 2000-01-01 -> 2024-08-13)
Adding data for BLDR: 79 out of 503
Adding data for BG: 80 out of 503
Adding data for BXP: 81 out of 503
Adding data for CDNS: 82 out of 503
Adding data for CZR: 83 out of 503
Adding data for CPT: 84 out of 503
Adding data for CPB: 85 out of 503
Adding data for COF: 86 out of 503
Adding data for CAH: 87 out of 503
Adding data for KMX: 88 out of 503
Adding data for CCL: 89 out of 503
Adding data for CARR: 90 out of 503
Adding data for CTLT: 91 out of 503
Adding data for CAT: 92 out of 503
Adding data for CBOE: 93 out of 503
Adding data for CBRE: 94 out of 503
Adding data for CDW: 95 out of 503
Adding data for CE: 96 out of 503
Adding data for COR: 97 out of 503
Adding data for CNC: 98 out of 503
Adding data for CNP: 99 out of 503
Adding data for CF: 100 out of 503
Adding data for CHRW: 101 out of 503
Adding data for CRL: 102 out of 503
Adding data for SCHW: 103 out of 503
Adding data for CHTR: 104 out of 503
Ad